In [1]:
import json
from typing import Any, Dict
from install_test.consts import REPO_SETS


DOCKERFILE_PATH = "resources/dataset/dockerfiles"
def load_repos():
    repos = {repo_set: json.load(open(repo_set, 'r')) for repo_set in REPO_SETS.values()}
    return repos

In [4]:
def calc_visbility(repo: Dict[str, Any]):
    # Calculate the "visibility" of a repo's documentation based on the following heursitic
    # 1 / (num directories to navigate + num files to find)
    dirs = [relevant.split('/') for relevant in repo['relevant_docs']]
    paths = [['/'.join(repo_dirs[:i+1]) for i in range(len(repo_dirs))] for repo_dirs in dirs]
    visibility = set([d for directories in paths for d in directories])
    return 1 / len(visibility) if len(visibility) > 0 else 0

In [5]:
repos = load_repos()
for repo in repos[REPO_SETS[-1]]:
    print(repo['url'])
    print(" - " + ', '.join(repo['relevant_docs']))
    visibility = calc_visbility(repo)
    repo['visibility'] = visibility
    print(" - " + str(visibility))


https://github.com/tiangolo/fastapi.git
 - docs/en/docs/contributing.md, docs/en/docs/tutorial/testing.md
 - 0.16666666666666666
https://github.com/psf/black.git
 - docs/contributing/the_basics.md
 - 0.3333333333333333
https://github.com/tqdm/tqdm.git
 - 
 - 0
https://github.com/Textualize/rich.git
 - CONTRIBUTING.md
 - 1.0
https://github.com/OpenInterpreter/open-interpreter.git
 - docs/CONTRIBUTING.md
 - 0.5
https://github.com/home-assistant/core.git
 - 
 - 0
https://github.com/sherlock-project/sherlock.git
 - 
 - 0
https://github.com/explosion/spaCy.git
 - README.md
 - 1.0
https://github.com/soimort/you-get.git
 - README.md
 - 1.0
https://github.com/Textualize/textual.git
 - CONTRIBUTING.md
 - 1.0


In [8]:
import os
from difflib import get_close_matches
from install_test.agent.functions import _get_file_contents, get_api_url

def get_dockerfile(repo, set_name):
    name = repo['url'].split('/')[-1][:-4]
    set_name = set_name.split('_')[-1][:-5]
    dockerfile_path = os.path.join(DOCKERFILE_PATH, set_name, name.lower()+'.dockerfile')
    if not os.path.exists(dockerfile_path):
        print(f"path {dockerfile_path} does not exist")
        return -2, []
    dockerfile = open(dockerfile_path, 'r').readlines()

    return dockerfile

def calc_informativity(repo: Dict[str, Any], set_name: str):
    # Calculate the "informativity" of a repo's documentation based on the following heuristic
    # (Num lines in model dockerfile that appear in documentation) / (Num lines in model dockerfile total)

    # step 1: load dockerfile
    dockerfile = get_dockerfile(repo, set_name)

    dockerfile = [
        ' '.join(line.split(' ')[1:]).strip()
        for line in dockerfile
        if not (
            line.startswith('COPY') or line.startswith('WORKDIR') or line.strip() == ''
        )
    ]

    # step 2: load documents
    if len(repo['relevant_docs']) == 0:
        return -1, []
    api_url = get_api_url(repo['url'])
    documents = [_get_file_contents(api_url, path, repo['ref']).split('\n') for path in repo['relevant_docs']]

    # step 3: find dockerfile lines that occur in documentation
    matched_lines = {}
    for doc, doc_name in zip(documents, repo['relevant_docs']):
        for line in doc:
            matches = get_close_matches(line, dockerfile, cutoff=0.8)
            if len(matches) > 0:
                if matches[0] not in matched_lines:
                    matched_lines[matches[0]] = []
                matched_lines[matches[0]].append((line, doc_name))
            else:
                for df_line in dockerfile:
                    if df_line in line:
                        if df_line not in matched_lines:
                            matched_lines[df_line] = []
                        matched_lines[df_line].append((line, doc_name))

    return (len(matched_lines) / len(dockerfile), matched_lines)

In [9]:
# CALCULATE BOTH HEURISTICS FOR ALL REPOS
for repo_set in REPO_SETS:
    for repo in repos[repo_set]:
        print(repo['url'], repo['ref'])
        print(" - " + ', '.join(repo['relevant_docs']))
        if 'info' not in repo or repo['info'] is None:
            info = calc_informativity(repo, repo_set)
            repo['info'] = info[0]
            print(" - informativity = " + str(info[0]))
        if 'visibility' not in repo or repo['visibility'] is None:
            visibility = calc_visbility(repo)
            repo['visibility'] = visibility
            print(" - visibility = " + str(visibility))

https://github.com/mandarons/icloud-drive-docker.git 8cbcc2c
 - 
https://github.com/typeddjango/django-stubs.git b325955
 - CONTRIBUTING.md
https://github.com/PennyLaneAI/pennylane.git b78565c
 - .github/CONTRIBUTING.md, doc/development/guide/installation.rst, doc/development/guide/tests.rst
https://github.com/CVHub520/X-AnyLabeling.git 7c62ffb
 - docs/en/get_started.md
https://github.com/open-compass/opencompass.git 7c7fa36
 - README.md, docs/en/get_started/installation.md
https://github.com/SciPhi-AI/R2R.git 712defb
 - docs/documentation/installation/local-system.mdx
https://github.com/VainF/Torch-Pruning.git 88e7472
 - README.md
https://github.com/scverse/scvi-tools.git e8f9b36
 - docs/developer/code.md
 - informativity = 0.3333333333333333
 - visibility = 0.3333333333333333
https://github.com/sabnzbd/sabnzbd.git 07250aa
 - README.md
https://github.com/dlt-hub/dlt.git 6451bd7
 - CONTRIBUTING.md
https://github.com/camel-ai/camel.git b8e1f5c
 - README.md, CONTRIBUTING.md, docs/get_sta

In [10]:
# WRITE HEURISTICS  
for repo_set in REPO_SETS:
    with open(repo_set, 'w') as f:
        json.dump(repos[repo_set], f)